In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install customized_konlpy

In [ ]:
from konlpy.tag import Komoran,Okt

In [ ]:
komoran = Komoran()
komoran.morphs('점심은부대찌개입니다')

In [ ]:
import pandas as pd
train = pd.read_csv("../input/train.csv")
train.head()

In [ ]:
train['smishing'].value_counts()
# 0과1의 비율이 6.3% 이기 때문에 validation도 함께 비율을 맞춰줘야한다.

In [ ]:
test = pd.read_csv("../input/public_test.csv")
test.head()

*딥러닝 텍스트 처리 중요한 것.

머신러닝, 딥러닝에서는 모든 데이터가  숫자로 바뀌어 있어야한다.(text든 이미지든,음성이든 예외 없이 모두다)

한 문장의 하나하나의 단어 기준을 세분화 하고 각각 숫자로 만들어 줘서 등록해야한다(이 과정이 look up table. ex) '고객님' : 1, 'OO은행' : 2 , 연락처:3

In [ ]:
# komoran.morphs(train['text'])
# # seires 형태로 되있어서 텍스트로 바꿔줘야함
# # apply 함수를 써서 적용함 (for문 느림)

In [ ]:
train['komoran_text']=train['text'].apply(lambda x : komoran.morphs(x))

In [ ]:
test['komoran_text']=test['text'].apply(lambda x : komoran.morphs(x))

In [ ]:
from keras.preprocessing.text import Tokenizer

tk = Tokenizer()
tk.fit_on_texts(list(train['komoran_text'])+list(test['komoran_text']))

tk.word_index

In [ ]:
train_text = tk.texts_to_sequences(train['komoran_text'])
test_text = tk.texts_to_sequences(test['komoran_text'])

In [ ]:
pd.Series(train_text).apply(len)

In [ ]:
len(tk.word_index)

In [ ]:
# 등록된 문자를 숫자로 바꿔줘야함(tk안에만등록)
train_text = tk.texts_to_sequences(train['text']) # train의 text를 숫자로 바꿔서 변수에 저장

test_text = tk.texts_to_sequences(test['text']) # test의 text를 숫자로 바꿔서 변수에 저장


In [ ]:
# 각각의 데이터에 칼럼의 갯수가 똑같아야지 모델이 학습을 시작할수있음.
# 가장 길 길이의 텍스트에 맞추면 됨.
# 여기선 가장큰 길이의 문자가 train의 317 이기때문에 317에 맞춰야함.

In [ ]:
# 1. 텍스트길이를 317 맥스값에 맞출것이아니라 잘라준다
# ->데이터가 날아가면 손해일것같지만 다른 데이터들이 많아서 괜찮다.
# -> 맥시멈값을 줄여주면 효율이좋아진다 -> 다른데이터들과의 길이 갭차이가 작아져서

In [ ]:
# 대부분의 텍스트대회의 데이터는 뒷쪽 데이터가 훨씬중요하다(한국말은 끝까지 들어봐야한다.)
# 뒷쪽의 데이터를 남겨두는게 중요하다.
# 기본값이 앞쪽의 데이터를 날리도록 되어있다.

In [ ]:
train[train['smishing'] == 1]['text']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
a,b = plt.subplots(1,1,figsize=(20,10)) #밑그림
sns.distplot(pd.Series(train_text).apply(len)) #분포표 displot
# 분포를 보고 length 값을 몇으로 할것인지 가장 효율적인지 보고 진행해준다

In [ ]:
pd.Series(train_text).apply(lambda x:len(x)).max()

In [ ]:
pd.Series(test_text).apply(lambda x:len(x)).max()

In [ ]:
train[train['smishing'] == 1]['text']

# 영어와는 다르게 한국말 특성상 뒤에 단어에 중요한 정보다 많이 있기때문에 앞에 문장들을 잘라서 사용했다(문자길이 100이상인것들만)
# 하지만 학습데이터의 라벨이 스미싱인('1') 데이터들만 모와서 봤더니, 맨 앞문장에 '(광고)'라는 키워드가 들어간걸 확인 할수있다.
# 그러므로 '(광고)' 키워드가 스미싱을 학습하는데 아주 중요한 역할을 하는데 없애면 안된다.
# 앞에 문장을 살리고 뒷쪽데이터만 날리는 학습파일(csv파일) 을 만들어서 두개 앙상블을 진행한다. 앞에 날린데이터와 뒤에날린데이터 모두 중요하기 떄문

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize =(20,12))
sns.distplot(pd.Series(train_text).apply(len))

In [ ]:
plt.figure(figsize =(20,12))
sns.distplot(pd.Series(test_text).apply(len))

In [ ]:
#모든 데이터의 단어의 갯수를 맞춰줘야함
from keras.preprocessing.sequence import pad_sequences

pad_train = pad_sequences(train_text, maxlen = 280)
pad_test = pad_sequences(test_text, maxlen =280)

In [ ]:
from keras import Sequential
from keras.layers import Dense,Embedding,Flatten 

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_vaild,y_train,y_vaild = train_test_split(pad_train,train['smishing'],random_state=2020,test_size=0.15,stratify=train['smishing'])
#random_state 값을 바꾸면서 val_loss가 제일 낮게 나오는것을 찾는다.

In [ ]:

model.add(Embedding(len(tk.word_index)+1,60,input_length=280))


model.add(Flatten())

model.add(Dense(2,activation ='softmax'))
#sigmoid를 쓰면 0으로 생성된다.

model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics= ['acc']) # 모델쌓기 끝났다고 선언해줘야함. 
# 분류모델을 해줄때는 y값을 분류로 one-hot-encoding을 해줘야하는데 안해줘서 회귀로 인식할수있어서 sparse 넣어준다

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint 

es = EarlyStopping(patience = 3) # 옵션이많다.알아보기, 이미지할때 loss가 크면 많이 참을수있도록 patience를 5이상으로 놓는다 텍스트는 안정적인편이여서 괜찮
mc = ModelCheckpoint('best.h5',save_best_only = True) #최적의 순간만 저장하겠다.

model.fit(x_train,y_train,epochs =20,validation_data=(x_vaild,y_vaild),batch_size=128,callbacks=[es,mc]) 

# 옵션값 1.은 학습횟수를 결정해줌 5번 반복횟수 2. 전체데이터셋의 10%만 학습에 이용하지않고 평가점수에만 이용함
# 학습에 이용하지 않는 이유는 overfitting.
# batch_size 는 속도조절(빠르게함)

#new
#학습에이용하지 않는 데이터를 정확한 비율로 (train_test_split에서 바꾼 값(stratify))

In [ ]:
# pre train embedding을 가져와서 우리 데이터셋에 적용해서 하면 가장 모델의 최적이 됨(위키피디아라다던지,페이스북이라던지)

In [ ]:
model.load_weights('best.h5')


In [ ]:
result = model.predict(pad_test,batch_size=128)


In [ ]:
test['smishing'] = result
sub = test[['id','smishing']]

In [ ]:
sub.to_csv('kb_competition_final.csv',index = False)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
pad_train = pad_sequences(train_text,maxlen = 100,truncating='post',padding='post')
pad_train
# 그래프를 보니 문자열 길이가 100 이하로만 설정해도 학습을 할 수 있다.(100이하인 문자열 데이터들이 매우많기때문에 효율적인 학습을 위해서 100보다 문자열이 긴 것들은 날려버린다.)
# truncating은 뒷쪽을 자르고, padding은 뒷쪽을 0으로 채운다

In [ ]:
pad_test = pad_sequences(test_text,maxlen = 100,truncating='post',padding='post')
pad_test

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_vaild,y_train,y_vaild = train_test_split(pad_train,train['smishing'],test_size = 0.15,random_state=33,stratify=train['smishing'])

In [ ]:
# 딥러닝 모델을 쌓는 방법 두가지꼭 필요함
from keras import Sequential # 1. 가볍고 쉽고 간단하게 하나씩 쌓는다
from keras.layers import Dense,Embedding,Flatten  
# 가장 많이 쓰는 층 불러온다 , # 텍스트는 임베드 층이 반드시있어야함 어떤 텍스트든,# 차원을 축소시킴(flatten)

In [ ]:
# 모델쌓기를 시작하겠습니다.
model = Sequential() # 객체생성

model.add(Embedding(448903,25,input_length=100)) 
# 입력층을 쌓겠다. 옵션 3가지 넣어야함(1. 전체데이터셋에대한 단어종류의 갯수(len(tk.word_index)),2.차원의 갯수를 설정,3.하나의 데이터당 몇개의 단어가 몇개가들어오는지 크기설정)

model.add(Flatten())
# 2차원인 애를 flatten 통해서 1차원으로 바꾸워줌(딥러닝에서는 네트워크 구조 연결이 중요하다)

model.add(Dense(5,activation = 'relu'))
model.add(Dense(1,activation ='sigmoid')) 
# 스미잉 인지 아닌지 출력이 되어야함(확률값으로)
# 출력층,옵션 2가지 (1.정답클래스의 갯수가 들어가야함, )
# 정답클래스의 갯수를 1을 넣었을경우엔(어떠한 확률) sigmoid 활성화 함수를, 2를 넣었을때는 softmax 활성화 함수를 넣어줘야함

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics= ['acc']) # 모델쌓기 끝났다고 선언해줘야함. 
# 옵션 3가지 
# 1.loss = 'sparse_categorycalcrossentropy'는 다중분류 썼을때 sparse_를쓰면 회귀가아닌 분류문제로 인식함(one-hot-encoding 할 필요없음)(필수)
# 2.optimizer = adam은학습효과 빠르고 학습 잘함 (필수)
# 3.metrics = ['acc]' <- 평가방식은 정확도로 보겠다.(선택)
# 딥러닝은 w 만 잘찾으면 됨!(어떻게? -> 손실함수값이 작아지는 방향으로 찾겠다.)



In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint 
# 자동으로 모델이 validation loss가 안좋아지면 이정도면 과적합이 일어나구 있구나 하고 자동으로 멈췸
# 모델이 최적의 순간을 저장해야되는데 학습시 어느순간 안좋아진다 그래서,최적의 가중치를 가지도록 
# 가장 점수가 잘 나올 가중치를 저장하고 예측에 사용한다.-> modelcheckpoint

es = EarlyStopping(patience = 3) #이미지할때 loss가 크면 많이 참을수있도록 patience를 5이상으로 놓는다 텍스트는 안정적인편이여서 괜찮
mc = ModelCheckpoint('best.h5',save_best_only = True) #최적의 순간만 저장하겠다.

In [ ]:
model.fit(pad_train,train['smishing'],epochs =20,validation_data = (x_vaild,y_vaild),batch_size=128,callbacks=[es,mc]) 

# 옵션값 1.은 학습횟수를 결정해줌 5번 반복횟수 2. 전체데이터셋의 10%만 학습에 이용하지않고 평가점수에만 이용함
# 학습에 이용하지 않는 이유는 overfitting.
# batch_size 는 속도조절(빠르게함)

In [ ]:
result = model.predict(pad_test,batch_size=128)

In [ ]:
result

In [ ]:
test['smishing'] = result
sub = test[['id','smishing']] #컬럼여러개로 접근할때 리스트괄호 2개
sub.to_csv('kb_competition3.csv',index = False)